<a href="https://colab.research.google.com/github/Ssactl/AH2179_Applied-AI-in-Transportation/blob/main/Module7_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from keras.models import load_model
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
filepath=""


In [2]:
#-------------------------------------------------data preprocessing------------------------------------------------------------------

# Define the URL of a CSV file containing data.
url = 'https://raw.githubusercontent.com/zhenliangma/Applied-AI-in-Transportation/master/Exercise_7_Neural_networks/Exercise7data.csv'

# Read the CSV data from the specified URL into a DataFrame (assuming you have the pandas library imported as 'pd').
df = pd.read_csv(url)

# Limit the DataFrame to the first 1000 rows (selecting a subset of the data).
df = df.iloc[:1000]

# Drop specific columns (Arrival_time, Stop_id, Bus_id, Line_id) from the DataFrame.
df = df.drop(['Arrival_time', 'Stop_id', 'Bus_id', 'Line_id'], axis=1)

# Extract the features (input variables) by dropping the 'Arrival_delay' column.
x = df.drop(['Arrival_delay'], axis=1)

# Extract the target variable ('Arrival_delay') as the variable to predict.
y = df['Arrival_delay']

# splite the train and test data
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [3]:
from itertools import product

layer_list=[1,2]

size_list=[32,64]
activ_list=['relu','sigmoid','tanh']
param_list=list(product(size_list,activ_list))
print(param_list)

[(32, 'relu'), (32, 'sigmoid'), (32, 'tanh'), (64, 'relu'), (64, 'sigmoid'), (64, 'tanh')]


In [4]:
import sys

NN_results=pd.DataFrame(columns=['layer1','layer2','mae','mse','r2'])

for layers in layer_list:
  model=Sequential()
  best_result={'layer1':None,'layer2':None,'layer3':None,'mae':sys.maxsize,'mse':sys.maxsize,'r2':-sys.maxsize-1}
  for param in product(param_list,repeat=layers):
    for l in range(0,layers):
      model.add(Dense(param[l][0], activation=param[l][1]))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mae', metrics=['mae'])
    hist = model.fit(X_train, y_train, validation_split=0.2, epochs=50, batch_size=32,verbose=0)
    # Use the trained model to predict on the test data.
    y_pred = model.predict(X_test)
    # Calculate Mean Absolute Error (MAE), Mean Squared Error (MSE), and R-squared (R2) for the predictions.
    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)

    if mse<best_result['mse']:
      for k in range(1,layers+1):
        best_result['layer'+str(k)]=param[k-1]
      best_result['mae']=mae
      best_result['mse']=mse
      best_result['r2']=r2

  print(layers)
  print(best_result)

7/7 [==============================] - 0s 3ms/step
1
{'layer1': (32, 'relu'), 'layer2': None, 'layer3': None, 'mae': 28.148973250389098, 'mse': 1431.4467803006328, 'r2': 0.8786702877550542}
7/7 [==============================] - 1s 5ms/step
2
{'layer1': (32, 'relu'), 'layer2': (32, 'relu'), 'layer3': None, 'mae': 28.104820039272308, 'mse': 1459.7287461463338, 'r2': 0.8762731027356715}
